In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import sqlite3
import mysql.connector as cnt
import plotly.express as px

import datetime as dt

# import mysql.connector as cnt
import sqlite3

import requests
from bs4 import BeautifulSoup

# import selenium
from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.firefox.options import Options as FirefoxOptions
# Firefox leistų sudėtingesnius nustatymus, bet neveikia per SNAP

from selenium import webdriver
from bs4 import BeautifulSoup
import time

print('Paskaita nr. 42, 2024-10-16 (pradėta 41, 2024-10-14)')
print('AutoPlius skelbimų nuorodas nuskaito į SQL3 duomenų bazę.')

Paskaita nr. 42, 2024-10-16 (pradėta 41, 2024-10-14)
AutoPlius skelbimų nuorodas nuskaito į SQL3 duomenų bazę.


In [124]:
# https://www.sqlitetutorial.net/sqlite-primary-key/
# https://www.crummy.com/software/BeautifulSoup/bs4/doc/

print('Sukuriama lentelė bazėje Testas.db')

sdb = sqlite3.connect('Testas.db')
cs = sdb.cursor()

sql_create_table = '''
create table if not exists auto_skelbimai
(
    nr integer primary key,
    href text
)
'''
cs.execute(sql_create_table)
# Reikia pridėti nuskaitymo tomestamp !!!

print('AutoPlius peržiūra ir įrašymas į db.')

for i in range(41,51):
    print()
    homeURL='https://m.autoplius.lt/skelbimai/naudoti-automobiliai?page_nr='+str(i)
    print(homeURL)

    opcijos = Options()
    opcijos.add_argument('--incognito')
    driver = webdriver.Chrome(options=opcijos)
    driver.get(homeURL)
    time.sleep(3) # 3 sekundėms atsidaro naršyklės langas
    source = driver.page_source
    driver.close()

    bs = BeautifulSoup(source,'html.parser')
    list_items=bs.find('div',{'class':'list-items'})
    automobiliai=list_items.find_all('a',{'class':'js-announcement-list-item'})
    print(f"Puslapyje rasta {len(automobiliai)} automobilių įrašų")

    for vienas in automobiliai:
        nr=int(vienas['href'][-13:-5])
        href=vienas['href']
        print(nr,href)

        if nr>10000000:
            sql_template='''INSERT OR REPLACE into auto_skelbimai (nr,href) values (?,?)'''
            cs.execute(sql_template,(nr,href))
        else:
            print('Netinkamas kodas',vienas['href'][-13:-5])

sdb.commit()
sdb.close()

print('\nAtlikta',dt.datetime.now().replace(microsecond=0))

Sukuriama lentelė bazėje Testas.db
AutoPlius peržiūra ir įrašymas į db.

https://m.autoplius.lt/skelbimai/naudoti-automobiliai?page_nr=41
Puslapyje rasta 20 automobilių įrašų
26500334 https://m.autoplius.lt/skelbimai/opel-astra-2-0-l-universalas-2011-dyzelinas-26500334.html
26881891 https://m.autoplius.lt/skelbimai/opel-astra-1-4-l-hecbekas-2010-benzinas-26881891.html
26881793 https://m.autoplius.lt/skelbimai/ford-s-max-2-0-l-vienaturis-2009-benzinas-26881793.html
26515632 https://m.autoplius.lt/skelbimai/bmw-318-2-0-l-universalas-2009-dyzelinas-26515632.html
26527014 https://m.autoplius.lt/skelbimai/bmw-320-2-0-l-universalas-2010-dyzelinas-26527014.html
26533768 https://m.autoplius.lt/skelbimai/toyota-avensis-2-0-l-universalas-2012-dyzelinas-26533768.html
26871563 https://m.autoplius.lt/skelbimai/saab-9-5-2-0-l-sedanas-2010-dyzelinas-26871563.html
26866555 https://m.autoplius.lt/skelbimai/volkswagen-passat-2-0-l-universalas-2012-dyzelinas-26866555.html
26785249 https://m.autoplius.lt/

In [ ]:
##################### Bandymai: #####################

In [53]:
homeURL = 'https://m.autoplius.lt/skelbimai/naudoti-automobiliai?page_nr=1'

opcijos = Options()
opcijos.add_argument('--incognito')
driver = webdriver.Chrome(options=opcijos)
driver.get(homeURL)
time.sleep(5) # 5 sekundėms atsidaro naršyklės langas
source = driver.page_source
driver.close()

bs = BeautifulSoup(source, 'html.parser')
list_items=bs.find('div',{'class':'list-items'})

print("list_items=bs.find('div',{'class':'list-items'})")
print(dt.datetime.now().replace(microsecond=0))

list_items=bs.find('div',{'class':'list-items'})
2024-10-16 20:52:19


In [119]:
print('Sukuriama lentelė Testas.db')

sdb = sqlite3.connect('Testas.db')
cs = sdb.cursor()

# https://www.sqlitetutorial.net/sqlite-primary-key/
# https://www.crummy.com/software/BeautifulSoup/bs4/doc/

sql_create_table = '''
create table if not exists auto_skelbimai
(
    nr integer primary key,
    href text
)
'''
cs.execute(sql_create_table)


Sukuriama lentelė Testas.db


In [ ]:

automobiliai=list_items.find_all('a',{'class':'js-announcement-list-item'}) # 'item-thumb'
print(f"Puslapyje rasta {len(automobiliai)} automobilių įrašų")
for vienas in automobiliai:
    nr=int(vienas['href'][-13:-5])
    href=vienas['href']
    print(nr,href)

    if nr>10000000:
        sql_template='''INSERT OR REPLACE into auto_skelbimai (nr,href) values (?,?)'''
        cs.execute(sql_template,(nr,href))
    else:
        print('Netinkamas kodas',vienas['href'][-13:-5])

sdb.commit()
sdb.close()

print('Atlikta',dt.datetime.now().replace(microsecond=0))
    

In [92]:
print(list_items.find('div',{'class':'paging'}))

<div class="paging">
<a class="fr btn button right-nav" href="/skelbimai/naudoti-automobiliai?page_nr=2">Pirmyn</a>
        
        1 / 200
    </div>


In [88]:
##################### Užrašai: #####################

# {'class': lambda x: x.startswith('ilgo pavadinimo pradžia')}
item_line=list_items.find('a',{'class':'js-announcement-list-item'})

# automobiliai=item_line.find_all('a')
# print(automobiliai[0],end='\n\n')
for a10 in automobiliai: # a10 = automobilis
    print(a10['href'])

# sk = bs.find('div', {'class':'GRID_ITEM'}).find('a')
# print(sk)
# print('::REM::')
# print(sk['href'])
# img = sk.find('img')
# print(img['height'])

# for a10 in automobiliai: # a10 = automobilis

a10=automobiliai[0]

# print(a10)
if a10['href'][23:32]=='skelbimai':
    # print(a10['class'],a10['href'])
    # print(a10,end='\n==============\n')
    gabalas=a10.find('div',{'class':'photo-gallery'})
    print(gabalas)
